# Step 1: Import & Load (and Sample)

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# 1. Load the Data
# We use low_memory=False because the dataset is mixed types in some columns
df_flights = pd.read_csv('flights.csv', low_memory=False)
df_airports = pd.read_csv('airports.csv')

# 2. Random Sample of 100,000 rows (for performance)
# We set a random_state so your group members get the exact same results every time.
df = df_flights.sample(n=100000, random_state=42)

print(f"Dataset Shape: {df.shape}")
df.head()

Dataset Shape: (100000, 31)


,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
84836,2015,1.0,6.0,2.0,DL,1190.0,N960DL,CAK,ATL,1115.0,...,1309.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
147509,2015,1.0,10.0,6.0,WN,3506.0,N748SW,RDU,TPA,1205.0,...,1348.0,-12.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
239209,2015,1.0,16.0,5.0,MQ,3225.0,N838MQ,DCA,JFK,1255.0,...,1422.0,8.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
141161,2015,1.0,9.0,5.0,DL,1667.0,N922DL,ATL,MYR,2102.0,...,2205.0,-5.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
104373,2015,1.0,7.0,3.0,US,1823.0,N939UW,PHL,PHX,1540.0,...,1917.0,13.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN


# Step 2: Data Cleaning

In [2]:
# --- CLEANING ---

# 1. Drop useless columns (Data Reduction)
# These columns usually have too many nulls or aren't needed for the dashboard
cols_to_drop = ['TAXI_OUT', 'TAXI_IN', 'WHEELS_OFF', 'WHEELS_ON', 'DIVERTED', 'CANCELLATION_REASON']
df = df.drop(columns=cols_to_drop)

# 2. Handle Missing Values
# If a flight was cancelled, it won't have a departure time. Let's drop rows where flight didn't happen.
df = df.dropna(subset=['DEPARTURE_TIME', 'ARRIVAL_DELAY'])

# 3. Fix Data Types
# Convert 'DATE' components into a proper datetime object
df['DATE'] = pd.to_datetime(df[['YEAR', 'MONTH', 'DAY']])

print("Missing values after cleaning:")
print(df.isnull().sum())

Missing values after cleaning:
YEAR                       0
MONTH                      0
DAY                        0
DAY_OF_WEEK                0
AIRLINE                    0
FLIGHT_NUMBER              0
TAIL_NUMBER                0
ORIGIN_AIRPORT             0
DESTINATION_AIRPORT        0
SCHEDULED_DEPARTURE        0
DEPARTURE_TIME             0
DEPARTURE_DELAY            0
SCHEDULED_TIME             0
ELAPSED_TIME               0
AIR_TIME                   0
DISTANCE                   0
SCHEDULED_ARRIVAL          0
ARRIVAL_TIME               0
ARRIVAL_DELAY              0
CANCELLED                  0
AIR_SYSTEM_DELAY       72915
SECURITY_DELAY         72915
AIRLINE_DELAY          72915
LATE_AIRCRAFT_DELAY    72915
WEATHER_DELAY          72915
DATE                       0
dtype: int64


# Step 3: Feature Engineering (3 Derived Attributes)

In [4]:
# --- DERIVED ATTRIBUTES ---

# Derived Attribute 1: 'Total_Delay'
# Sum of departure and arrival delay (negative numbers mean early, which helps averages)
df['Total_Delay'] = df['DEPARTURE_DELAY'] + df['ARRIVAL_DELAY']

# Derived Attribute 2: 'Delay_Status' (Categorical)
# Useful for Pie Charts/Composition analysis
def categorize_delay(minutes):
    if minutes <= 0:
        return 'On Time / Early'
    elif minutes < 30:
        return 'Minor Delay'
    else:
        return 'Major Delay'

df['Delay_Status'] = df['Total_Delay'].apply(categorize_delay) # Note: Ensure column case matches

# Derived Attribute 3: 'Route'
# Useful for identifying busy paths (e.g., "JFK -> LHR")
df['Route'] = df['ORIGIN_AIRPORT'].astype(str) + " -> " + df['DESTINATION_AIRPORT'].astype(str)

print("New attributes created: Total_Delay, Delay_Status, Route")

New attributes created: Total_Delay, Delay_Status, Route


# Step 4: Merging (Crucial for Maps)

In [5]:
# Merge for Origin Airport Coordinates
df = df.merge(df_airports, left_on='ORIGIN_AIRPORT', right_on='IATA_CODE', how='left')

# Rename columns to avoid confusion (LATITUDE_x vs LATITUDE_y)
df = df.rename(columns={'LATITUDE': 'Origin_Lat', 'LONGITUDE': 'Origin_Long', 'CITY': 'Origin_City'})

# Drop duplicate key column
df = df.drop(columns=['IATA_CODE'])

print("Data Merged. Ready for visualization.")

Data Merged. Ready for visualization.


In [6]:
df.to_csv('cleaned_flight_data.csv', index=False)